In [10]:
pip install ratelimiter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import pandas as pd
import numpy as np
import requests
from time import sleep
import datetime
from ratelimiter import RateLimiter
from google.colab import drive

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
#creation de la liste earoports europe dans lesquelles lufthansa opère
airports = pd.read_csv("airports.csv", sep = ";")
airports["Continent"] = airports["TimeZoneId"].apply(lambda x : x.split("/")[0])
europe = airports[airports['Continent']== "Europe"].reset_index(drop=True)["AirportCode"]

In [ ]:
#creation des listes individuels 
np.random.seed(123)

ousmane , abdoul , abdel = np.array_split(np.random.permutation(europe), 3)

In [15]:
#collecte de flight_information_departures
def get_flight_information_departures(origins, date, access_tokens):
    flights = []

    rate_limiter = RateLimiter(max_calls=4, period=1)  # 4 requêtes par seconde
    api_key_index = 0
    requests_count = 0

    for origin in origins:
        for hour in range(24):
            for minute in ["00", "30"]:
                with rate_limiter:
                    if requests_count >= 550:
                        print(f"Changement de clé API : utilisation de la clé {api_key_index + 1}") 
                        api_key_index += 1
                        requests_count = 0
                        if api_key_index >= len(access_tokens):
                            print("Toutes les clés API ont atteint la limite de requêtes.")
                            return pd.DataFrame.from_dict(flights).drop_duplicates().reset_index(drop=True)

                    time_str = f"{hour:02d}:{minute}"
                    datetime = f"{date}T{time_str}"
                    
                    authentification = {"Authorization": "Bearer " + access_tokens[api_key_index]}

                    try:
                        response = requests.get(
                            ("https://api.lufthansa.com/v1/operations/customerflightinformation/departures/{origin}/{datetime}"
                             "?limit=100&offset=0"
                            ).format(origin=origin, datetime=datetime),
                            headers=authentification).json()

                        for flight in response['FlightInformation']['Flights']['Flight']:
                            # Extraction des données du vol 
                            departure = flight.get('Departure', {})
                            arrival = flight.get('Arrival',{})
                            operatingCarrier = flight.get('OperatingCarrier')
                            equipment = flight.get('Equipment')
                            status = flight.get('Status')


                            row = {
                                'DpAirportCode': departure.get('AirportCode'),
                                'DpScheduledDate': departure.get('Scheduled', {}).get('Date'),
                                'DpScheduledTime': departure.get('Scheduled', {}).get('Time'),
                                'DpActualDate': departure.get('Actual', {}).get('Date'),
                                'DpActualTime': departure.get('Actual', {}).get('Time'),
                                'DpTerminalName': departure.get('Terminal', {}).get('Name'),
                                'DpTerminalGate': departure.get('Terminal', {}).get('Gate'),
                                'DpStatusCode': departure.get('Status', {}).get('Code'),
                                'DPStatusDescription': departure.get('Status', {}).get('Description'),
                                'AirlineID': operatingCarrier.get('AirlineID'),
                                'FlightNumber': operatingCarrier.get('FlightNumber'),
                                'AircraftCode': equipment.get('AircraftCode'),
                                'ArrActualDate':arrival.get('Actual', {}).get('Date'),
                                'ArrActualTime':arrival.get('Actual', {}).get('Time'),
                                'ArrTerminalName':arrival.get('Terminal', {}).get('Name'),
                                'ArrTerminalGate':arrival.get('Terminal', {}).get('Gate'),
                                'ArrStatusCode':arrival.get('Status', {}).get('Code'),
                                'ArrStatusDescription':arrival.get('Status', {}).get('Description'),
                                'StatusCode' : status.get('Code'),
                                'StatusDescription' : status.get('Description')

                            }

                            flights.append(row)

                        requests_count += 1

                    except Exception as e:
                        print(f"Error for origin {origin}, time {time_str}: {e}")

    flight_information_departures = pd.DataFrame.from_dict(flights).reset_index(drop = True)
    flight_information_departures = flight_information_departures.drop_duplicates().reset_index(drop = True)
    flight_information_departures.insert(0, 'FlightId', flight_information_departures['AirlineID'] + flight_information_departures['FlightNumber'].astype(str))
    return flight_information_departures

access_tokens = ["z395y299xm7fjrzxmydrynyc", "apj5gpxj9c9ut9up6g68gsxs",
                 "5a5zsvd83ym6nb4mz47ddjds", "s7j65m93x5ap6ne7qfk4dkva",
                 "4hv9m2n4epq83z7j3g5j8ne8", "mucjtvh5vnbvuhpsktepesns",
                 "ztwk3etpxtvbnzmq5ex7vq8t", "fvzvx8ksbyrvgz4xnyzd8qtk",]

origins = ["LHR","CDG","AMS","FRA","MAD","BCN","MUC","LGW","FCO","DUB","ORY"]
date = "2023-05-19"

flight_information_departures = get_flight_information_departures(origins, date, access_tokens)
flight_information_departures.to_csv("/content/drive/MyDrive/flight_information_departures_europe.csv", sep = ";", index=False)
display(flight_information_departures)


Error for origin LHR, time 00:00: 'FlightInformation'
Error for origin LHR, time 00:30: 'FlightInformation'
Error for origin LHR, time 01:00: 'FlightInformation'
Error for origin LHR, time 01:30: 'FlightInformation'
Error for origin LHR, time 11:00: [Errno Expecting value] <error>
  <category>Server</category>
  <text>APPL0014: IO Exception: Read timed out
java.net.SocketTimeoutException: Read timed out</text>
</error>
: 0
Error for origin LHR, time 21:00: 'FlightInformation'
Error for origin LHR, time 21:30: 'FlightInformation'
Error for origin LHR, time 22:00: 'FlightInformation'
Error for origin LHR, time 22:30: 'FlightInformation'
Error for origin LHR, time 23:00: 'FlightInformation'
Error for origin LHR, time 23:30: 'FlightInformation'
Error for origin CDG, time 00:00: 'FlightInformation'
Error for origin CDG, time 00:30: 'FlightInformation'
Error for origin CDG, time 01:00: 'FlightInformation'
Error for origin CDG, time 01:30: 'FlightInformation'
Error for origin CDG, time 02:00:

,FlightId,DpAirportCode,DpScheduledDate,DpScheduledTime,DpActualDate,DpActualTime,DpTerminalName,DpTerminalGate,DpStatusCode,DPStatusDescription,...,FlightNumber,AircraftCode,ArrActualDate,ArrActualTime,ArrTerminalName,ArrTerminalGate,ArrStatusCode,ArrStatusDescription,StatusCode,StatusDescription
0,LX345,LHR,2023-05-19,06:00,2023-05-19,05:56,2,A17,DP,Flight Departed,...,345,32N,2023-05-19,08:30,None,None,LD,Flight Landed,LD,Flight Landed
1,OS458,LHR,2023-05-19,06:00,2023-05-19,05:52,2,B42,DP,Flight Departed,...,458,32N,2023-05-19,08:57,3,G16A,LD,Flight Landed,LD,Flight Landed
2,LH921,LHR,2023-05-19,06:30,2023-05-19,06:38,2,A26,DP,Flight Departed,...,921,32N,2023-05-19,09:08,1,B25,LD,Flight Landed,LD,Flight Landed
3,SN2104,LHR,2023-05-19,06:50,2023-05-19,07:07,2,None,DP,Flight Departed,...,2104,320,2023-05-19,09:00,None,None,LD,Flight Landed,LD,Flight Landed
4,LH2483,LHR,2023-05-19,06:55,2023-05-19,06:57,2,A18,DP,Flight Departed,...,2483,32N,2023-05-19,09:48,2,H20,LD,Flight Landed,LD,Flight Landed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,EW9395,DUB,2023-05-19,12:45,2023-05-19,12:36,1,None,DP,Flight Departed,...,9395,31D,2023-05-19,15:15,None,A79,LD,Flight Landed,LD,Flight Landed
935,LH981,DUB,2023-05-19,17:30,2023-05-19,17:59,1,302,DP,Flight Departed,...,981,321,2023-05-19,20:55,1,B46,LD,Flight Landed,LD,Flight Landed
936,4Y2519,DUB,2023-05-19,17:45,2023-05-19,18:14,1,306,DP,Flight Departed,...,2519,32A,2023-05-19,21:35,2,H10,LD,Flight Landed,LD,Flight Landed
937,2L405,DUB,2023-05-19,19:00,2023-05-19,19:06,1,302,DP,Flight Departed,...,405,295,2023-05-19,22:14,None,None,LD,Flight Landed,LD,Flight Landed


In [16]:
#collecte de flight_information_arrivals
def get_flight_information_arrivals(origins, date, access_tokens):
    flights = []

    rate_limiter = RateLimiter(max_calls=4, period=1)  # 4 requêtes par seconde
    api_key_index = 0
    requests_count = 0

    for origin in origins:
        for hour in range(24):
            for minute in ["00", "30"]:
                with rate_limiter:
                    if requests_count >= 750:
                        print(f"Changement de clé API : utilisation de la clé {api_key_index + 1}") 
                        api_key_index += 1
                        requests_count = 0
                        if api_key_index >= len(access_tokens):
                            print("Toutes les clés API ont atteint la limite de requêtes.")
                            return pd.DataFrame.from_dict(flights).drop_duplicates().reset_index(drop=True)

                    time_str = f"{hour:02d}:{minute}"
                    datetime = f"{date}T{time_str}"
                    
                    authentification = {"Authorization": "Bearer " + access_tokens[api_key_index]}

                    try:
                        response = requests.get(
                            ("https://api.lufthansa.com/v1/operations/customerflightinformation/arrivals/{origin}/{datetime}"
                             "?limit=100&offset=0"
                            ).format(origin=origin, datetime=datetime),
                            headers=authentification).json()

                        for flight in response['FlightInformation']['Flights']['Flight']:
                            # Extraction des données du vol
                            arrival = flight.get('Arrival',{})
                            departure = flight.get('Departure', {})
                            operatingCarrier = flight.get('OperatingCarrier')
                            equipment = flight.get('Equipment')
                            status = flight.get('Status')

                            row = {
                                'ArrAirportCode': arrival.get('AirportCode'),
                                'ArrScheduledDate': arrival.get('Scheduled', {}).get('Date'),
                                'ArrScheduledTime': arrival.get('Scheduled', {}).get('Time'),
                                'ArrActualDate': arrival.get('Actual', {}).get('Date'),
                                'ArrActualTime': arrival.get('Actual', {}).get('Time'),
                                'ArrTerminalName': arrival.get('Terminal', {}).get('Name'),
                                'ArrTerminalGate': arrival.get('Terminal', {}).get('Gate'),
                                'ArrStatusCode': arrival.get('Status', {}).get('Code'),
                                'ArrStatusDescription': arrival.get('Status', {}).get('Description'),
                                'AirlineID': operatingCarrier.get('AirlineID'),
                                'FlightNumber': operatingCarrier.get('FlightNumber'),
                                'AircraftCode': equipment.get('AircraftCode'),
                                'DpActualDate':departure.get('Actual', {}).get('Date'),
                                'DpActualTime':departure.get('Actual', {}).get('Time'),
                                'DpTerminalName':departure.get('Terminal', {}).get('Name'),
                                'DpTerminalGate':departure.get('Terminal', {}).get('Gate'),
                                'DpStatusCode':departure.get('Status', {}).get('Code'),
                                'DpStatusDescription':departure.get('Status', {}).get('Description'),
                                'StatusCode' : status.get('Code'),
                                'StatusDescription' : status.get('Description')

                            }
                            flights.append(row)

                        requests_count += 1

                    except Exception as e:
                        print(f"Error for origin {origin}, time {time_str}: {e}")

    flight_information_arrivals = pd.DataFrame.from_dict(flights).reset_index(drop = True)
    flight_information_arrivals = flight_information_arrivals.drop_duplicates().reset_index(drop = True)
    flight_information_arrivals.insert(0, 'FlightId', flight_information_arrivals['AirlineID'] + flight_information_arrivals['FlightNumber'].astype(str))
    return flight_information_arrivals

access_tokens = ["m2e8jrjgfywaah8xpjxgttmm", "apj5gpxj9c9ut9up6g68gsxs",
                 "5a5zsvd83ym6nb4mz47ddjds", "s7j65m93x5ap6ne7qfk4dkva",
                 "4hv9m2n4epq83z7j3g5j8ne8", "mucjtvh5vnbvuhpsktepesns",
                 "ztwk3etpxtvbnzmq5ex7vq8t", "fvzvx8ksbyrvgz4xnyzd8qtk",]

origins = ["LHR","CDG","AMS","FRA","MAD","BCN","MUC","LGW","FCO","DUB","ORY"]
date = "2023-05-19"

flight_information_arrivals = get_flight_information_arrivals(origins, date, access_tokens)
flight_information_arrivals.to_csv("/content/drive/MyDrive/flight_information_arrivals.csv", sep = ";" , index=False)
display(flight_information_arrivals)

Error for origin LHR, time 00:00: 'FlightInformation'
Error for origin LHR, time 00:30: 'FlightInformation'
Error for origin LHR, time 01:00: 'FlightInformation'
Error for origin LHR, time 01:30: 'FlightInformation'
Error for origin LHR, time 02:00: 'FlightInformation'
Error for origin LHR, time 02:30: 'FlightInformation'
Error for origin LHR, time 03:00: 'FlightInformation'
Error for origin LHR, time 03:30: 'str' object has no attribute 'get'
Error for origin LHR, time 22:00: 'str' object has no attribute 'get'
Error for origin LHR, time 22:30: 'str' object has no attribute 'get'
Error for origin LHR, time 23:00: 'FlightInformation'
Error for origin LHR, time 23:30: 'FlightInformation'
Error for origin CDG, time 00:00: 'FlightInformation'
Error for origin CDG, time 00:30: 'FlightInformation'
Error for origin CDG, time 01:00: 'FlightInformation'
Error for origin CDG, time 01:30: 'FlightInformation'
Error for origin CDG, time 02:00: 'FlightInformation'
Error for origin CDG, time 02:30: 

,FlightId,ArrAirportCode,ArrScheduledDate,ArrScheduledTime,ArrActualDate,ArrActualTime,ArrTerminalName,ArrTerminalGate,ArrStatusCode,ArrStatusDescription,...,FlightNumber,AircraftCode,DpActualDate,DpActualTime,DpTerminalName,DpTerminalGate,DpStatusCode,DpStatusDescription,StatusCode,StatusDescription
0,EW9468,LHR,2023-05-19,07:25,2023-05-19,07:19,2,None,LD,Flight Landed,...,9468,31D,2023-05-19,06:57,None,C46,DP,Flight Departed,LD,Flight Landed
1,LH924,LHR,2023-05-19,07:40,2023-05-19,07:40,2,None,LD,Flight Landed,...,924,32N,2023-05-19,06:59,1,B28,DP,Flight Departed,LD,Flight Landed
2,LX316,LHR,2023-05-19,07:55,2023-05-19,07:58,2,None,LD,Flight Landed,...,316,32N,2023-05-19,07:10,None,None,DP,Flight Departed,LD,Flight Landed
3,EW7464,LHR,2023-05-19,08:05,2023-05-19,08:07,2,None,LD,Flight Landed,...,7464,31D,2023-05-19,07:21,1,B47,DP,Flight Departed,LD,Flight Landed
4,EW460,LHR,2023-05-19,08:10,2023-05-19,08:18,2,None,LD,Flight Landed,...,460,320,2023-05-19,07:47,1,D82,DP,Flight Departed,LD,Flight Landed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,EW9394,DUB,2023-05-19,12:05,2023-05-19,11:40,1,None,LD,Flight Landed,...,9394,31D,2023-05-19,11:03,None,A80,DP,Flight Departed,LD,Flight Landed
932,LH980,DUB,2023-05-19,16:45,2023-05-19,16:35,1,None,LD,Flight Landed,...,980,321,2023-05-19,15:43,1,B32,DP,Flight Departed,LD,Flight Landed
933,4Y2518,DUB,2023-05-19,17:00,2023-05-19,17:12,1,None,LD,Flight Landed,...,2518,32A,2023-05-19,15:53,2,H38,DP,Flight Departed,LD,Flight Landed
934,2L404,DUB,2023-05-19,18:15,2023-05-19,17:57,1,None,LD,Flight Landed,...,404,295,2023-05-19,16:47,None,None,DP,Flight Departed,LD,Flight Landed
